In [1]:
using Pkg

Pkg.activate(".")

  Activating project at `~/Documents/Uni/Master/MA/visual_analysis`


In [2]:
include("utils.jl")
include("visual_utils.jl")

    CondaPkg Found dependencies: /home/denis/Documents/Uni/Master/MA/visual_analysis/CondaPkg.toml
    CondaPkg Found dependencies: /home/denis/.julia/packages/PythonCall/bb3ax/CondaPkg.toml
    CondaPkg Dependencies already up to date


show_eof_modes_of_timeline_pyeof (generic function with 1 method)

In [3]:
using GLMakie
using GeoMakie
using EmpiricalOrthogonalFunctions
using NCDatasets 
using Dates
using BenchmarkTools
using Statistics

In [4]:
mon_mean_base_path = "/home/denis/workspace/data/ivt_monthly_mean"
day_mean_base_path = "/home/denis/workspace/data/ivt_daily_mean"

"/home/denis/workspace/data/ivt_daily_mean"

In [5]:
# monthly_tl_data = filter_by_date(build_timeline_data(mon_mean_base_path, 10, "ssp126", "ssp585"; file_range_selection=:)) do timedata
#     return year(timedata) >= 2081 && year(timedata) <= 2100  
# end

In [6]:
winter_months = [12, 1, 2, 3]


4-element Vector{Int64}:
 12
  1
  2
  3

In [8]:

monthly_tl_data = filter_by_date(build_timeline_data(mon_mean_base_path, 10, "ssp126", "ssp585"; file_range_selection = :)) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-16T12:00:00"), DateTime("2015-02-14T21:00:00"), DateTime("2015-03-16T09:00:00"), DateTime("2015-12-16T09:00:00"), DateTime("2016-01-16T09:00:00"), DateTime("2016-02-15T09:00:00"), DateTime("2016-03-16T09:00:00"), DateTime("2016-12-16T09:00:00"), DateTime("2017-01-16T09:00:00"), DateTime("2017-02-14T21:00:00")  …  DateTime("2098-12-16T09:00:

In [7]:
daily_tl_data = filter_by_date(build_timeline_data(day_mean_base_path, 10, "ssp126", "ssp585"; file_range_selection = :)) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-01T12:00:00"), DateTime("2015-01-02T09:00:00"), DateTime("2015-01-03T09:00:00"), DateTime("2015-01-04T09:00:00"), DateTime("2015-01-05T09:00:00"), DateTime("2015-01-06T09:00:00"), DateTime("2015-01-07T09:00:00"), DateTime("2015-01-08T09:00:00"), DateTime("2015-01-09T09:00:00"), DateTime("2015-01-10T09:00:00")  …  DateTime("2100-12-23T09:00:

In [9]:
# truth_data = filter_by_date(build_timeline_data(base_path, 10, "ssp126", "ssp585"; file_range_selection=1:2)) do timedata
#     return year(timedata) >= 2015 && year(timedata) <= 2034
# end

In [36]:
compare_truth_with_tldata(truth_data, tl_data, "mon_mean_comparison.mp4"; framerate = 3, resolution = (2000, 1000), colormap = :managua)

In [8]:
show_eof_modes_of_timeline(daily_tl_data, 4, 18, "eof_modes_daily_18seasons_aligned.mp4"; colormap = :broc, framerate = 2, resolution = (2000, 1100), eof_center = true, whole_figure_title = "Leading EOF Modes of Daily Averages of IVT (18 Winter Period) Aligned")

Threads available: 8
Handled scope 6398:8581 out of 69 on thread 4
Handled scope 2153:4335 out of 69 on thread 7
Handled scope 5428:7611 out of 69 on thread 6
Handled scope 1061:3244 out of 69 on thread 1
Handled scope 4336:6519 out of 69 on thread 8
Handled scope 1:2152 out of 69 on thread 5
Handled scope 7369:9551 out of 69 on thread 3
Handled scope 3245:5427 out of 69 on thread 2
Handled scope 2274:4456 out of 69 on thread 7
Handled scope 6520:8702 out of 69 on thread 4
Handled scope 1183:3365 out of 69 on thread 1
Handled scope 91:2273 out of 69 on thread 5
Handled scope 4457:6640 out of 69 on thread 8
Handled scope 3366:5549 out of 69 on thread 2
Handled scope 7491:9672 out of 69 on thread 3
Handled scope 5550:7732 out of 69 on thread 6
Handled scope 2395:4578 out of 69 on thread 7
Handled scope 5671:7853 out of 69 on thread 6
Handled scope 7612:9794 out of 69 on thread 3
Handled scope 1304:3486 out of 69 on thread 1
Handled scope 6641:8823 out of 69 on thread 4
Handled scope 3487